In [ ]:
!pip install openai scikit-learn pillow

In [ ]:
from openai import OpenAI
import json
import base64
import requests
import PIL.Image
import numpy as np
from sklearn.metrics import roc_auc_score

In [ ]:
client = OpenAI(api_key='<openai-api-key>')

In [ ]:
### Download link for dataset
https://uark-my.sharepoint.com/:u:/g/personal/haovan_uark_edu/EU3T0FTfNTtBhKNNMo0ZHW0BEn1ZouV9wd1SrcSkW9csBw?e=MESUhd

In [ ]:
with open("/home/haovan/hateful_memes/test_unseen" + '.jsonl', 'r') as file:
    test_data_meta = list(file)

img_dir = '/home/haovan/hateful_memes/'
    


In [ ]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
def scale_image(image_path, new_height = 150):
    img = PIL.Image.open(image_path)
    ratio = img.size[1] / new_height
    new_width = img.size[0] / ratio
    img = img.resize((int(new_width), int(new_height)))
    img.save(img_dir+"/scaled_img/"+test_sample['img'].split("/")[-1]) 

In [ ]:
for idx, json_str in enumerate(test_data_meta):
    test_sample = json.loads(json_str)
    image_path = img_dir+test_sample['img']
    scale_image(image_path, new_height=150)

In [ ]:
# prompt = lambda ocr_text: f"""Hatefulness definition: Hate speech is strictly defined as a direct or indirect attack on people based on characteristics, including ethnicity, race, nationality, immigration status, religion, caste, sex, gender identity, sexual orientation, disability, or disease. The definition of attack includes violent or dehumanizing speech, statements of inferiority, and calls for exclusion or segregation. Mocking hate crime is also considered hate speech.
# Classification criteria: A meme is considered hateful if either the image or text is hateful on its own. A meme is also considered hateful if both the image and text are not hateful individually, but together convey a hateful message. If neither the image nor text is hateful, and they don't create a hateful meaning together, the meme is not hateful.

# You consider visual and textual meanings to classify the meme as hateful or non-hateful based on the hatefulness definition and classification criteria.
# The text written in the image: "{ocr_text}"
# After giving a short explanation, provide the answer with following format:
# Classification:
# Probability of the meme being hateful (from 0 to 1):"""

In [ ]:
prompt = lambda ocr_text: f"""Hatefulness definition: Hate speech is strictly defined as a direct or indirect attack on people based on characteristics, including ethnicity, race, nationality, immigration status, religion, caste, sex, gender identity, sexual orientation, disability, or disease. The definition of attack includes violent or dehumanizing speech, statements of inferiority, and calls for exclusion or segregation. Mocking hate crime is also considered hate speech.
Classification criteria: A meme is considered hateful if either the image or text is hateful on its own. A meme is also considered hateful if both the image and text are not hateful individually, but together convey a hateful message. If neither the image nor text is hateful, and they don't create a hateful meaning together, the meme is not hateful.

You consider visual and textual meanings to classify the meme as hateful or non-hateful based on the hatefulness definition and classification criteria.
After giving a short explanation, provide the answer with following format:
Classification:
Probability of the meme being hateful (from 0 to 1):
"""

In [ ]:
last_processed_idx = -1
unprocessed_ids = []
responses = []
full_responses = []

In [ ]:
for idx, json_str in enumerate(test_data_meta):
    if idx <= last_processed_idx:
        continue
    test_sample = json.loads(json_str)
    image_path = img_dir+"/scaled_img/"+test_sample['img'].split("/")[-1]

    base64_image = encode_image(image_path)
    print(f"Processing image: {test_sample['img']}, actual label: {test_sample['label']}")

    try:
        response = client.chat.completions.create(
          model="gpt-4o",
          messages=[
            {
              "role": "user",
              "content": [
                {
                    "type": "text", 
                    "text": prompt(test_sample['text'])
                },
                {
                  "type": "image_url",
                  "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}",
                  },
                },
              ],
            }
          ],
          max_tokens=300,
        )
        answer = response.choices[0].message.content
        full_responses.append(response.choices[0])
    except Exception as e:
        unprocessed_ids.append(test_sample['img'])
        answer = "ERROR WITH REQUEST"
        full_responses.append(e)
        
    responses.append(answer)
    last_processed_idx = idx
    print(answer)
    print()
    


In [ ]:
print(f"Number of responses: {len(responses)}")
print(f"Number of failed responses: {len(unprocessed_ids)}")

In [ ]:
from datetime import datetime
import pickle
now_time = datetime.now()

file_name  = f'results_gpt4o_unseen_test_{now_time.strftime("%H_%M_%S__%d_%m_%Y")}.pkl'
with open(file_name, 'wb') as f:
    pickle.dump({"responses": responses, "unprocessed_ids": unprocessed_ids}, f)

In [ ]:
with open(file_name, 'rb') as f:
    data = pickle.load(f)
    responses = data['responses']

In [ ]:
import re

actuals = []
predictions = []
prob_list = []
threshold = 0.5
for idx, json_str in enumerate(test_data_meta):
    test_sample = json.loads(json_str)
    actual_class = test_sample['label']
    actuals.append(actual_class)

    lower_response = responses[idx].lower()
    all_found_vals = re.findall("probability of the meme being hateful.*?\:.*?\n?[0-9]+\.?[0-9]*.*?", lower_response)
    if len(all_found_vals) == 0:
        is_hateful = False
        hateful_keywords = ["classification: hateful", "classified as hateful"]
        print(f'>>>>> Could not find probability in response - Idx: {idx} - Res: {lower_response}')
        for kw in hateful_keywords:
            if kw in lower_response:
                is_hateful = True
                break
        if is_hateful:
            predicted_class = 1
            prob = 1.0
        else:
            predicted_class = 0
            prob = 0.0
        print(f'Assigned probability: {prob} <<<<<\n\n' )
    else:
        num_str = re.sub('[^0-9\.]', '', (all_found_vals[0].split(" "))[-1])
        if num_str[-1] == '.':
            num_str = num_str[:-1]
        if float(num_str) >= threshold:
            predicted_class = 1
        else:
            predicted_class = 0
        prob = float(num_str)
    prob_list.append(prob)
    predictions.append(predicted_class)



In [ ]:
acc = np.mean(np.array(actuals) == np.array(predictions))
print(f"Accuracy: {acc}")
print(f"AUROC: {roc_auc_score(actuals, predictions)}")

In [ ]:
unprocessed_ids